In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten , Dense
from tensorflow.keras.models import Model
from tensorflow.nn import relu , sigmoid
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os
import urllib
import zipfile
import shutil

In [6]:
file_names = []
classes = []
root_path = '../input/fruit-and-vegetable-image-recognition/train'
for folder in os.listdir(root_path):
    new_path = os.path.join(root_path,folder)
    for file in os.listdir(new_path):
        file_names.append(os.path.join(new_path,file))
        classes.append(folder)

data = pd.DataFrame({'filenames':file_names,'class':classes})

In [10]:
data['class'].value_counts()

In [12]:
train_data_gen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_data_gen = ImageDataGenerator(rescale=1./255)

In [15]:
train_data = train_data_gen.flow_from_directory(
    '../input/fruit-and-vegetable-image-recognition/train',
    batch_size=100 ,
    target_size=(150,150),
    class_mode='binary'
)

test_data = test_data_gen.flow_from_directory(
    '../input/fruit-and-vegetable-image-recognition/test' ,
    batch_size=100 ,
    target_size=(150,150),
    class_mode='binary'
)

In [18]:
pre_trained_model = InceptionV3(include_top=False , weights=None , input_shape=(150,150,3))
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')

x = Flatten()(last_layer.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

In [20]:
model.compile(
    loss=tf.keras.losses.binary_crossentropy ,
    optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
    metrics=['acc']
)

In [ ]:
history = model.fit(
    train_data ,
    epochs=2 ,
    verbose = 1,
    validation_data=test_data ,
)

In [23]:
_ , ax = plt.subplots(1,2,figsize = (16,5))
ax[0].plot(history.history['acc'])
ax[0].plot(history.history['val_acc'])
ax[0].set_title("Accurecy")

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title("Loss")

In [27]:
cab_img = tf.keras.preprocessing.image.load_img('../input/fruit-and-vegetable-image-recognition/train/cabbage/Image_11.jpg' , target_size=(150,150,3))
cab_img = tf.keras.preprocessing.image.img_to_array(cat_img)
cab_img = cat_img / 255.0
plt.imshow(cat_img)

import tensorflow as tf


In [28]:
train_data.class_indices

In [ ]:
model.predict(cab_img[tf.newaxis , ])